In [ ]:
%reload_ext autoreload
%autoreload 2


import functools
print = functools.partial(print, flush=True)

import os
import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
import matplotlib.pyplot as plt
from matplotlib import cm
from pathlib import Path
import pickle
from tqdm import tqdm
import scipy

import flexiznam as flz
from cottage_analysis.analysis import spheres, common_utils
from cottage_analysis.pipelines import pipeline_utils
from v1_depth_analysis.v1_manuscript_2023 import depth_selectivity, get_session_list
from v1_depth_analysis.v1_manuscript_2023 import common_utils as plt_common_utils

In [ ]:
VERSION = 1
SAVE_ROOT = '/camp/lab/znamenskiyp/home/shared/presentations/v1_manuscript_2023/ver'+str(VERSION)+'/fig1/'
os.makedirs(SAVE_ROOT, exist_ok=True)

In [ ]:
# Load data
# Load an example session
project='hey2_3d-vision_foodres_20220101'
session_name='PZAH8.2h_S20230116'
flexilims_session = flz.get_flexilims_session(project)

vs_df_example, trials_df_example = spheres.sync_all_recordings(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={'anatomical_only':3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
)

neurons_ds_example = pipeline_utils.create_neurons_ds(
    session_name=session_name, flexilims_session=flexilims_session, project=None, conflicts="skip"
)
neurons_df_example = pd.read_pickle(neurons_ds_example.path_full)



In [ ]:
# Get PSTH for all sessions
project='hey2_3d-vision_foodres_20220101'
flexilims_session = flz.get_flexilims_session(project)
session_list = get_session_list.get_session_list()
results_all = depth_selectivity.get_psth_crossval_all_sessions(flexilims_session, session_list, nbins=10, closed_loop=1)

In [ ]:
# Concatenate neurons_df for all sessions
neurons_df_all = plt_common_utils.concatenate_all_neurons_df(flexilims_session, session_list=get_session_list.get_session_list(), read_iscell=True, verbose=True)


In [ ]:
fig = plt.figure()

# Fig.1A: Motion parallax schema & equation, VR schema
# Fig.1B: Trial structure schema, Spheres with different sizes schema

# Fig.1C: Raster plot of an example neuron
EXAMPLE_ROI = 250
depth_selectivity.plot_raster_all_depths(
    fig=fig,
    neurons_df=neurons_df_example,
    trials_df=trials_df_example,
    roi=EXAMPLE_ROI,
    is_closed_loop=True,
    max_distance=6,
    nbins=60,
    frame_rate=15,
    vmax=3,
    plot=True,
    plot_x=0,
    plot_y=1,
    plot_width=0.8,
    plot_height=0.2,
    cbar_width=0.01,
    fontsize_dict={'title':15, 'label':10, 'tick':10},
    );

# Fig.1D: Example PSTH of a single neuron
depth_selectivity.plot_PSTH(fig=fig,
            trials_df=trials_df_example,
            roi=EXAMPLE_ROI,
            is_closed_loop=True,
            max_distance=6,
            nbins=20,
            frame_rate=15,
            plot_x=0.85,
            plot_y=1,
            plot_width=0.2,
            plot_height=0.2,
            fontsize_dict={'title':15, 'label':10, 'tick':10, 'legend':5},
            linewidth=2,
            legend_on=True,
            )

# Fig.1E: Example depth tuning curve & PSTH of a single neuron
depth_selectivity.plot_depth_tuning_curve(
    fig=fig,
    neurons_df=neurons_df_example,
    trials_df=trials_df_example,
    roi=250,
    rs_thr=0.2,
    plot_fit=False,
    linewidth=3,
    linecolor="k",
    fit_linecolor="r",
    closed_loop=1,
    plot_x=0,
    plot_y=0.67,
    plot_width=0.2,
    plot_height=0.2,
    fontsize_dict={'title':15, 'label':10, 'tick':10},
    )

# Fig.1F: Example depth tuning curve & PSTH of 3 other neurons
EXAMPLE_ROIS = [249, 54, 88]
for i, roi in enumerate(EXAMPLE_ROIS):
    if i==len(EXAMPLE_ROIS)-1:
        legend_on=True
    else:
        legend_on=False
    depth_selectivity.plot_PSTH(fig=fig,
            trials_df=trials_df_example,
            roi=roi,
            is_closed_loop=True,
            max_distance=6,
            nbins=20,
            frame_rate=15,
            plot_x=0.05+0.25*(i+1),
            plot_y=0.67,
            plot_width=0.2,
            plot_height=0.18,
            fontsize_dict={'title':15, 'label':10, 'tick':10, 'legend':5},
            linewidth=2,
            legend_on=legend_on,
            )
    if i!=0:
        plt.ylabel('')
    
    depth_selectivity.plot_depth_tuning_curve(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=0.2,
        plot_fit=False,
        linewidth=2,
        linecolor="k",
        fit_linecolor="r",
        closed_loop=1,
        plot_x=0.05+0.25*(i+1)+0.12,
        plot_y=0.79,
        plot_width=0.08,
        plot_height=0.08,
        fontsize_dict={'title':15, 'label':10, 'tick':10},
        )
    plt.ylabel('')
    plt.xlabel('')
    plt.xticks([])
        
# Fig.1G: Histogram of preferred depths of all depth-tuned neurons; Raster plot of preferred depths of all neurons (sorted by preferred depth)
depth_selectivity.plot_preferred_depth_hist(
    fig=fig,
    results_df=results_all,
    use_col="preferred_depth_crossval",
    nbins=50,
    plot_x=0,
    plot_y=0.2,
    plot_width=0.4,
    plot_height=0.3,
    fontsize_dict={"title": 15, "label": 10, "tick": 10},
)

depth_selectivity.plot_psth_raster(
    fig=fig,
    results_df=results_all,
    depth_list=np.geomspace(5,640,num=8).astype(int),
    use_cols=["preferred_depth_crossval", "psth_crossval", "preferred_depth_rsq"],
    depth_rsq_thr = 0.04,
    plot_x=0.6,
    plot_y=0.2,
    plot_width=0.4,
    plot_height=0.3,
    fontsize_dict={"title": 15, "label": 10, "tick": 10},
)

# example session of depth preference distribution on 2p image
# histogram of % of depth-tuned neurons in each session
# re-assess the threshold using significant test: spearman r significance test for 5-folded cross validation (prediction vs actual response)
# Plot same raster for running vs. not-running


# Size tuning plot
# look at r2 for all cells size-tuned vs depth-tuned

# Visually responsive neurons
# 0.5s before the onset vs average of the whole trial (look at mean response)
fig.savefig(SAVE_ROOT+'fig1.pdf', bbox_inches='tight', transparent=True)

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as mpimg

# Load the image
img = mpimg.imread('plot_image.png')

# Create a figure
fig, ax = plt.subplots()

# Display the image
ax.imshow(img)
ax.axis('off')  # Turn off axis

plt.show()

In [ ]:
# Depth tuning curve & PSTH of different running states
fig=plt.figure()
rois = [250, 249, 54, 88]
for i, roi in enumerate(rois):
    # depth tuning curve
    ax=depth_selectivity.plot_depth_tuning_curve(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=0.2,
        plot_fit=False,
        linewidth=2,
        linecolor="r",
        fit_linecolor="r",
        closed_loop=1,
        plot_x=0+0.25*(i+1),
        plot_y=1,
        plot_width=0.2,
        plot_height=0.2,
        label="Running (> 20cm/s)",
        fontsize_dict={'title':15, 'label':10, 'tick':10},
        )
    if i!=0:
        plt.ylabel('')
    
    depth_selectivity.plot_depth_tuning_curve(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=None,
        still_only=False,
        plot_fit=False,
        linewidth=2,
        linecolor="b",
        fit_linecolor="r",
        closed_loop=1,
        overwrite_ax=False,
        ax=ax,
        plot_x=0+0.25*(i+1),
        plot_y=1,
        plot_width=0.2,
        plot_height=0.2,
        label="All",
        fontsize_dict={'title':15, 'label':10, 'tick':10},
        )
    if i!=0:
        plt.ylabel('')
        
    depth_selectivity.plot_depth_tuning_curve(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=0.05,
        rs_thr_max=0.2,
        still_only=False,
        plot_fit=False,
        linewidth=2,
        linecolor="orange",
        fit_linecolor="r",
        closed_loop=1,
        overwrite_ax=False,
        ax=ax,
        plot_x=0+0.25*(i+1),
        plot_y=1,
        plot_width=0.2,
        plot_height=0.2,
        label="Walking (5-20cm/s)",
        fontsize_dict={'title':15, 'label':10, 'tick':10},
        )
    if i!=0:
        plt.ylabel('')
        
    depth_selectivity.plot_depth_tuning_curve(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=None,
        rs_thr_max=0.05,
        still_only=True,
        still_time=1,
        frame_rate=15,
        plot_fit=False,
        linewidth=2,
        linecolor="gray",
        fit_linecolor="r",
        closed_loop=1,
        overwrite_ax=False,
        ax=ax,
        plot_x=0+0.15*(i+1),
        plot_y=1,
        plot_width=0.2,
        plot_height=0.2,
        label="Not running for 1s",
        fontsize_dict={'title':15, 'label':10, 'tick':10},
        )
    if i!=0:
        plt.ylabel('')
    if i==len(rois)-1:
        plt.legend(fontsize=5)
    
    #PSTH
    depth_selectivity.plot_PSTH(fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=True,
        max_distance=6,
        nbins=20,
        frame_rate=15,
        rs_thr_min=0.2,
        still_only=False,
        still_time=0,
        plot_x=0+0.25*(i+1),
        plot_y=0.47,
        plot_width=0.15,
        plot_height=0.1,
        fontsize_dict={'title':15, 'label':10, 'tick':10, 'legend':5},
        linewidth=2,
        legend_on=False,
        )
    if i!=0:
        plt.ylabel('')
    if i==0:
        plt.title("Running (> 20cm/s)")
    ylim = plt.gca().get_ylim()
    plt.xlabel('')
    
    depth_selectivity.plot_PSTH(fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=True,
        max_distance=6,
        nbins=20,
        frame_rate=15,
        rs_thr_min=None,
        still_only=False,
        still_time=0,
        plot_x=0+0.25*(i+1),
        plot_y=0.7,
        plot_width=0.15,
        plot_height=0.1,
        fontsize_dict={'title':15, 'label':10, 'tick':10, 'legend':5},
        linewidth=2,
        legend_on=False,
        )
    if i!=0:
        plt.ylabel('')
    if i==0:
        plt.title("All frames")
    plt.ylim(ylim)
    plt.xlabel('')
    
    depth_selectivity.plot_PSTH(fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=True,
        max_distance=6,
        nbins=20,
        frame_rate=15,
        rs_thr_min=0.05,
        rs_thr_max=0.2,
        still_only=False,
        still_time=0,
        plot_x=0+0.25*(i+1),
        plot_y=0.24,
        plot_width=0.15,
        plot_height=0.1,
        fontsize_dict={'title':15, 'label':10, 'tick':10, 'legend':5},
        linewidth=2,
        legend_on=False,
        )
    if i!=0:
        plt.ylabel('')
    if i==0:
        plt.title("Walking (5-20cm/s)")
    plt.ylim(ylim)
    plt.xlabel('')
        
    depth_selectivity.plot_PSTH(fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=True,
        max_distance=6,
        nbins=20,
        frame_rate=15,
        rs_thr_min=None,
        rs_thr_max=0.05,
        still_only=True,
        still_time=1,
        plot_x=0+0.25*(i+1),
        plot_y=0,
        plot_width=0.15,
        plot_height=0.1,
        fontsize_dict={'title':15, 'label':10, 'tick':10, 'legend':5},
        linewidth=2,
        legend_on=False,
        )
    if i!=0:
        plt.ylabel('')
    if i==0:
        plt.title("Not running for 1s")
    plt.ylim(ylim)
    
# QUANTIFY: select depth-tuned neurons based on all frames, compare max response from running / non-running
# CHANGE not-running to look into the past for 1s
# Plot walking (5-20cm/s)

In [ ]:
# Proportion of depth-tuned neurons in each session
rsq_thr = 0.04
neurons_df_all = neurons_df_all[neurons_df_all['iscell'] ==1]
neuron_sum = neurons_df_all.groupby('session')[['roi']].agg(['count']).values.flatten()
prop = neurons_df_all.groupby('session').apply(lambda x: x[x['depth_tuning_test_rsq_closedloop'] > rsq_thr][['roi']].agg(['count'])).values.flatten()
plt.subplot(121)
plt.hist(prop/neuron_sum, bins=50)

rsq_thr = 0.01
neurons_df_all = neurons_df_all[neurons_df_all['iscell'] ==1]
neuron_sum = neurons_df_all.groupby('session')[['roi']].agg(['count']).values.flatten()
prop = neurons_df_all.groupby('session').apply(lambda x: x[x['depth_tuning_test_rsq_closedloop'] > rsq_thr][['roi']].agg(['count'])).values.flatten()
plt.subplot(122)
plt.hist(prop/neuron_sum, bins=50)



In [ ]:
neurons_df_all_original = neurons_df_all.copy()

In [ ]:
np.mean(neurons_df_all["depth_tuning_test_rsq_closedloop"] > 0.01)